In [1]:
import lxml.etree as etree
import os
     
def indent(elem, level=0):
    tab = "    "
    i = "\n" + level*tab
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + tab
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            
def parseIndent(inpath, outpath, xmlfile):
    parser = etree.XMLParser(resolve_entities=False, strip_cdata=False, recover=True)
    document = etree.parse(os.path.join(inpath,xmlfile), parser)
    root = document.getroot()
    indent(root)
    
    Query = root.find(".//{http://www.areva.com/DataBindingMap2.xsd}Query")
    if Query is not None:
        qroot = etree.XML(bytes(bytearray(Query.text, encoding='utf-8')))
        indent(qroot, level=5)
        Query.text=etree.tostring(qroot, encoding='utf-8', xml_declaration=True)
        
    Values = root.findall(".//Value")
    if Values:
        for Value in Values:
            vroot = etree.XML(bytes(bytearray(Value.text, encoding='utf-16')))
            indent(vroot, level=6)
            Value.text=etree.tostring(vroot, encoding='utf-8', xml_declaration=True)
            
    if not os.path.isdir(outpath): os.mkdir(outpath)
    document.write(os.path.join(outpath,xmlfile), pretty_print=True, encoding='utf-8', xml_declaration=True, standalone='yes')
    
def parseIgrids(inpath,outpath):
    for root, dirs, files in os.walk(inpath, topdown=False):
        for file in files:
            if file.endswith(".grid"):
                parseIndent(root, os.path.join(outpath,os.path.basename(root)), file)

In [ ]:
X = parseIndent("C:\DATA\Engineering_Support\Tools\EMS\local_display\mydisplays\displays\smtnet","./igrid_cleaned/","smtnet_mtos_ems1.grid")

In [ ]:
Y = parseIgrids("C:/DATA/Source/rtes/igrid","./igrid_cleaned/")

In [ ]:
Z = parseIgrids("C:\DATA\Engineering_Support\Tools\EMS\local_display\mydisplays\displays","./igrid_cleaned/")

In [2]:
Z = parseIgrids("C:/DATA/Source/rtes/ems/psa/","./igrid_cleaned/")